In [2]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

In [3]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5

no_cuda = False
log_interval = 100

In [ ]:
# tensorflow : (28, 28, 1)
# pytorch : (1, 28, 28)

In [5]:
# Model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
# Prepocess

import os

In [8]:
sorted(os.listdir('../../data/dataset/mnist_png/training/'))

['.DS_Store', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [9]:
train_dir = '../../data/dataset/mnist_png/training'
test_dir = '../../data/dataset/mnist_png/testing'

In [11]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [12]:
train_dataset = datasets.ImageFolder(root=train_dir,
                                    transform=transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.1307,), (0.3081,))
                                    ]))

test_dataset = datasets.ImageFolder(root=test_dir,
                                   transform=transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.1307,), (0.3081,))
                                   ]))


In [13]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [14]:
# Optimization

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [16]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()    # backpropagation 계산하기 전에 0으로 기울기 계산
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
                  
    # Test Mode
    model.eval()    # batch norm이나 dropout 등을 train mode 변환
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:    # autograd engine, 즉 backpropagation 이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()    # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)    # get the index of the max log-probabillity
            correct += pred.eq(target.view_as(pred)).sum().item()    # pred와 target과 같은지 확인
            
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.076938
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.722558
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.331144
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.587047
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.409746
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.555121
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.524024
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.675466
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.428546
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.215653

Test set: Average loss: 0.3227, Accuracy: 9075/10000 (91%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.392473
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.310485
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.467251
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.380238
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.417933
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.167031
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.447830
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.289363
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.255886
T